# Perovskite Solution Calculator

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt

# Create raw data directly in the code (from labBook_simplified.xlsx)
raw_data_dict = {
    'Chemical': ['MAI', 'FAI', 'MABr', 'CsI', 'PbI2', 'PbBr2', '2PACz', 'RbI', 'PbCl2', 'MACl'],
    'Mol_Weight': [158.97, 171.97, 111.97, 259.81, 461.01, 367.01, 275.24, 212.37, 278.11, 67.52],
    'Density': [3.01597, 2.604, 2.167, 4.51, 7.392, 8.7, 1.0, 3.11, 5.85, 1.21]
}
raw_data = pd.DataFrame(raw_data_dict).set_index('Chemical')

# Create a class to handle the calculator
class PerovskiteCalculator:
    def __init__(self):
        self.create_layout()
        
    def create_layout(self):
        # Create a grid layout that matches the Excel sheet
        # Increased grid size to accommodate all rows
        self.grid = widgets.GridspecLayout(35, 15, width='100%')
        
        # Add title
        self.title = widgets.HTML(
            value='<h1 style="text-align: center;">Perovskite Solution Calculator</h1>',
            layout=widgets.Layout(grid_area='1/1/2/16')
        )
        self.grid[0, :] = self.title
        
        # Add legend
        self.legend = widgets.HTML(
            value='''
            <div style="padding: 10px; background-color: #f8f8f8; border: 1px solid #ddd;">
                <h3>Legend:</h3>
                <p><span style="background-color: #ffcccc; padding: 2px 5px;">Change</span> - Editable fields</p>
                <p><span style="background-color: #ccffcc; padding: 2px 5px;">Copy</span> - Copy from another cell</p>
                <p><span style="background-color: #ccccff; padding: 2px 5px;">Constant</span> - Fixed values</p>
            </div>
            ''',
            layout=widgets.Layout(grid_area='2/10/5/15')
        )
        self.grid[1:4, 10:14] = self.legend
        
        # Experiment number (K1 in Excel)
        self.exp_num_label = widgets.HTML(
            value='<div style="text-align: right; padding: 5px;"><b>Experiment #:</b></div>',
            layout=widgets.Layout(width='auto')
        )
        self.grid[1, 0] = self.exp_num_label
        
        self.exp_num = widgets.IntText(
            value=1,
            layout=widgets.Layout(width='100px', background_color='#ffcccc')
        )
        self.grid[1, 1] = self.exp_num
        
        # Density input
        self.density_label = widgets.HTML(
            value='<div style="text-align: right; padding: 5px;"><b>Density (g/mL):</b></div>',
            layout=widgets.Layout(width='auto')
        )
        self.grid[7, 0] = self.density_label
        
        self.density = widgets.FloatText(
            value=0.978,
            layout=widgets.Layout(width='100px', background_color='#ccccff')
        )
        self.grid[7, 1] = self.density
        
        self.density_unit = widgets.HTML(
            value='<div style="padding: 5px;">g/mL</div>',
            layout=widgets.Layout(width='auto')
        )
        self.grid[7, 2] = self.density_unit
        
        # Formulation header
        self.formulation_header = widgets.HTML(
            value='<div style="padding: 5px;"><b>Formulation</b></div>',
            layout=widgets.Layout(width='auto')
        )
        self.grid[9, 0] = self.formulation_header
        
        # Chemical headers
        chemicals = ['PbI2', 'PbBr2', 'CsI', '2PACz', '', 'FAI', 'MABr']
        for i, chem in enumerate(chemicals):
            self.grid[10, i+1] = widgets.HTML(
                value=f'<div style="text-align: center; padding: 5px;"><b>{chem}</b></div>',
                layout=widgets.Layout(width='auto')
            )
        
        # Molar ratio header and inputs
        self.ratio_header = widgets.HTML(
            value='<div style="padding: 5px;"><b>Molar Ratio:</b></div>',
            layout=widgets.Layout(width='auto')
        )
        self.grid[10, 11] = self.ratio_header
        
        self.ratio_pbi2 = widgets.BoundedIntText(
            value=83,
            min=0,
            max=100,
            layout=widgets.Layout(width='60px', background_color='#ffcccc')
        )
        self.grid[10, 12] = self.ratio_pbi2
        
        self.ratio_pbbr2 = widgets.BoundedIntText(
            value=17,
            min=0,
            max=100,
            layout=widgets.Layout(width='60px', background_color='#ffcccc')
        )
        self.grid[10, 13] = self.ratio_pbbr2
        
        self.ratio_csi = widgets.BoundedIntText(
            value=5,
            min=0,
            max=100,
            layout=widgets.Layout(width='60px', background_color='#ffcccc')
        )
        self.grid[10, 14] = self.ratio_csi
        
        # Row headers
        row_headers = [
            'Flask name', 'Mol. weight (g/mol)', 'Density (g/cm³)', 'Goal(mol/L)',
            'g/L', 'Goal mL', 'Goal g', 'Achieved g', 'Achieved mL', 'Density (g/mL)',
            'Ratio', 'Achieved(mol/L)', 'PB Salt mL', 'Solvent mL', 'Solvent', 'End (g/L)', 'Total Vol'
        ]
        
        for i, header in enumerate(row_headers):
            self.grid[i+11, 0] = widgets.HTML(
                value=f'<div style="text-align: right; padding: 5px;"><b>{header}</b></div>',
                layout=widgets.Layout(width='auto')
            )
        
        # Create the data grid
        self.cells = {}
        chemicals = ['PbI2', 'PbBr2', 'CsI', '2PACz', 'FAI', 'MABr']
        
        # Flask name inputs (K5)
        for i, chem in enumerate(chemicals):
            col = i+1 if i < 4 else i+2  # Skip the empty column
            self.cells[f'{chem}_flask'] = widgets.Text(
                value=f'{chem}_1',
                layout=widgets.Layout(width='100px', background_color='#ffcccc')
            )
            self.grid[11, col] = self.cells[f'{chem}_flask']
        
        # Molecular weight (display only)
        for i, chem in enumerate(chemicals):
            col = i+1 if i < 4 else i+2
            mw = raw_data.loc[chem, 'Mol_Weight'] if chem in raw_data.index else 0
            self.cells[f'{chem}_mw'] = widgets.HTML(
                value=f'<div style="padding: 5px; text-align: center;">{mw}</div>',
                layout=widgets.Layout(width='auto')
            )
            self.grid[12, col] = self.cells[f'{chem}_mw']
        
        # Density (display only)
        for i, chem in enumerate(chemicals):
            col = i+1 if i < 4 else i+2
            density = raw_data.loc[chem, 'Density'] if chem in raw_data.index else 0
            self.cells[f'{chem}_density'] = widgets.HTML(
                value=f'<div style="padding: 5px; text-align: center;">{density}</div>',
                layout=widgets.Layout(width='auto')
            )
            self.grid[13, col] = self.cells[f'{chem}_density']
        
        # Goal mol/L inputs (K6)
        default_concs = {'PbI2': 1.5, 'PbBr2': 1.5, 'CsI': 1.5, '2PACz': 0.0033, 'FAI': 1.24, 'MABr': 1.24}
        for i, chem in enumerate(chemicals):
            col = i+1 if i < 4 else i+2
            self.cells[f'{chem}_goal_mol'] = widgets.FloatText(
                value=default_concs[chem],
                layout=widgets.Layout(width='100px', background_color='#ffcccc')
            )
            self.grid[14, col] = self.cells[f'{chem}_goal_mol']
        
        # Create empty cells for calculated values
        for row in range(15, 28):  # g/L through Total Vol
            for i, chem in enumerate(chemicals):
                col = i+1 if i < 4 else i+2
                cell_id = f'{chem}_row{row}'
                self.cells[cell_id] = widgets.HTML(
                    value='<div style="padding: 5px; text-align: center;">-</div>',
                    layout=widgets.Layout(width='auto')
                )
                self.grid[row, col] = self.cells[cell_id]
        
        # Add solvent types
        solvent_types = {'PbI2': '4:1', 'PbBr2': '4:1', 'CsI': 'DMSO', '2PACz': 'EtOH', 
                        'FAI': '4:1 + PbI', 'MABr': '4:1 + PbBr'}
        for i, chem in enumerate(chemicals):
            col = i+1 if i < 4 else i+2
            self.cells[f'{chem}_solvent'] = widgets.HTML(
                value=f'<div style="padding: 5px; text-align: center;">{solvent_types[chem]}</div>',
                layout=widgets.Layout(width='auto')
            )
            self.grid[24, col] = self.cells[f'{chem}_solvent']
        
        # Add x value input
        self.x_label = widgets.HTML(
            value='<div style="text-align: right; padding: 5px;"><b>x:</b></div>',
            layout=widgets.Layout(width='auto')
        )
        self.grid[28, 0] = self.x_label
        
        self.x_value = widgets.FloatText(
            value=4.0,
            layout=widgets.Layout(width='100px', background_color='#ffcccc')
        )
        self.grid[28, 1] = self.x_value
        
        # Add summary section
        self.summary_header = widgets.HTML(
            value='<div style="padding: 10px;"><h3>Formulation Summary:</h3></div>',
            layout=widgets.Layout(width='auto')
        )
        self.grid[11, 11:15] = self.summary_header
        
        # Summary labels
        summary_labels = ['PbI2+FAI:', 'PbBr2+MABr:', 'CsI:', 'Total Volume:']
        for i, label in enumerate(summary_labels):
            self.grid[12+i, 11] = widgets.HTML(
                value=f'<div style="padding: 5px; text-align: right;"><b>{label}</b></div>',
                layout=widgets.Layout(width='auto')
            )
        
        # Summary values
        self.summary_values = {}
        for i, key in enumerate(['pbi2_fai', 'pbbr2_mabr', 'csi', 'total']):
            self.summary_values[key] = widgets.HTML(
                value='<div style="padding: 5px;">-</div>',
                layout=widgets.Layout(width='auto')
            )
            self.grid[12+i, 12] = self.summary_values[key]
        
        # Add calculate button
        self.calculate_button = widgets.Button(
            description='Calculate',
            button_style='success',
            layout=widgets.Layout(width='150px')
        )
        self.calculate_button.on_click(self.calculate)
        self.grid[16, 11] = self.calculate_button
        
        # Add chart area
        self.output = widgets.Output()
        self.grid[17:24, 11:15] = self.output
        
    def calculate(self, b):
        with self.output:
            clear_output()
            
            # Get x value (multiplier)
            x = self.x_value.value
            
            # Get ratios
            ratio_pbi2 = self.ratio_pbi2.value / 100
            ratio_pbbr2 = self.ratio_pbbr2.value / 100
            ratio_csi = self.ratio_csi.value / 100
            
            # Calculate for each chemical
            chemicals = ['PbI2', 'PbBr2', 'CsI', '2PACz', 'FAI', 'MABr']
            goal_mls = {}
            goal_gs = {}
            achieved_mols = {}
            
            for chem in chemicals:
                # Get molecular weight and density
                mw = float(raw_data.loc[chem, 'Mol_Weight']) if chem in raw_data.index else 0
                density = float(raw_data.loc[chem, 'Density']) if chem in raw_data.index else 0
                
                # Get goal mol/L
                goal_mol = self.cells[f'{chem}_goal_mol'].value
                
                # Calculate g/L
                g_per_l = goal_mol * mw
                self.cells[f'{chem}_row15'].value = f'<div style="padding: 5px; text-align: center;">{g_per_l:.3f}</div>'
                
                # Calculate goal mL based on ratios
                if chem == 'PbI2':
                    goal_ml = ratio_pbi2 * x
                elif chem == 'PbBr2':
                    goal_ml = ratio_pbbr2 * x
                elif chem == 'CsI':
                    goal_ml = ratio_csi * x
                elif chem == '2PACz':
                    goal_ml = x
                elif chem == 'FAI':
                    goal_ml = ratio_pbi2 * x
                else:  # MABr
                    goal_ml = ratio_pbbr2 * x
                
                goal_mls[chem] = goal_ml
                self.cells[f'{chem}_row16'].value = f'<div style="padding: 5px; text-align: center;">{goal_ml:.4f}</div>'
                
                # Calculate goal g
                goal_g = g_per_l * goal_ml / 1000
                goal_gs[chem] = goal_g
                self.cells[f'{chem}_row17'].value = f'<div style="padding: 5px; text-align: center;">{goal_g:.4f}</div>'
                
                # Set achieved g (same as goal g)
                self.cells[f'{chem}_row18'].value = f'<div style="padding: 5px; text-align: center;">{goal_g:.4f}</div>'
                
                # Set achieved mL (same as goal mL)
                self.cells[f'{chem}_row19'].value = f'<div style="padding: 5px; text-align: center;">{goal_ml:.4f}</div>'
                
                # Set solution density
                solution_density = 1.51 if chem == 'PbI2' else 1.42
                self.cells[f'{chem}_row20'].value = f'<div style="padding: 5px; text-align: center;">{solution_density:.2f}</div>'
                
                # Set ratio (1.09 for Pb compounds, 1 for others)
                ratio = 1.09 if chem in ['PbI2', 'PbBr2', 'CsI', '2PACz'] else 1.0
                self.cells[f'{chem}_row21'].value = f'<div style="padding: 5px; text-align: center;">{ratio:.2f}</div>'
                
                # Calculate achieved mol/L
                achieved_mol = goal_g / (goal_ml / 1000) / mw
                achieved_mols[chem] = achieved_mol
                self.cells[f'{chem}_row22'].value = f'<div style="padding: 5px; text-align: center;">{achieved_mol:.4f}</div>'
                
                # Calculate PB Salt mL
                if chem in ['FAI', 'MABr']:
                    pb_salt_ml = goal_ml * (0.995 if chem == 'FAI' else 0.97)
                    self.cells[f'{chem}_row23'].value = f'<div style="padding: 5px; text-align: center;">{pb_salt_ml:.4f}</div>'
                else:
                    self.cells[f'{chem}_row23'].value = '<div style="padding: 5px; text-align: center;">-</div>'
                
                # Calculate Solvent mL
                if chem in ['FAI', 'MABr']:
                    pb_salt_ml = goal_ml * (0.995 if chem == 'FAI' else 0.97)
                    solvent_ml = goal_ml - pb_salt_ml
                else:
                    solvent_ml = goal_ml
                
                self.cells[f'{chem}_row24'].value = f'<div style="padding: 5px; text-align: center;">{solvent_ml:.4f}</div>'
                
                # Set end g/L (same as g/L)
                self.cells[f'{chem}_row25'].value = f'<div style="padding: 5px; text-align: center;">{g_per_l:.3f}</div>'
                
                # Calculate total volume
                if chem in ['PbI2', 'FAI']:
                    total_vol = goal_mls['PbI2'] + (goal_mls['FAI'] - (goal_mls['FAI'] * 0.995))
                elif chem in ['PbBr2', 'MABr']:
                    total_vol = goal_mls['PbBr2'] + (goal_mls['MABr'] - (goal_mls['MABr'] * 0.97))
                else:
                    total_vol = goal_ml
                
                self.cells[f'{chem}_row26'].value = f'<div style="padding: 5px; text-align: center;">{total_vol:.4f}</div>'
            
            # Update summary values
            pbi2_fai_ul = goal_mls['PbI2'] * 1000
            pbbr2_mabr_ul = goal_mls['PbBr2'] * 1000
            csi_ul = goal_mls['CsI'] * 1000
            total_ul = pbi2_fai_ul + pbbr2_mabr_ul + csi_ul
            
            self.summary_values['pbi2_fai'].value = f'<div style="padding: 5px;">{pbi2_fai_ul:.1f} µL</div>'
            self.summary_values['pbbr2_mabr'].value = f'<div style="padding: 5px;">{pbbr2_mabr_ul:.1f} µL</div>'
            self.summary_values['csi'].value = f'<div style="padding: 5px;">{csi_ul:.1f} µL</div>'
            self.summary_values['total'].value = f'<div style="padding: 5px;">{total_ul:.1f} µL</div>'
            
            # Create a pie chart of the formulation
            plt.figure(figsize=(5, 4))
            plt.pie([pbi2_fai_ul, pbbr2_mabr_ul, csi_ul], 
                   labels=[f'PbI2+FAI\n{pbi2_fai_ul:.1f} µL', 
                           f'PbBr2+MABr\n{pbbr2_mabr_ul:.1f} µL', 
                           f'CsI\n{csi_ul:.1f} µL'],
                   autopct='%1.1f%%',
                   colors=['#ff9999', '#66b3ff', '#99ff99'])
            plt.title(f'Formulation Composition (Experiment #{self.exp_num.value})')
            plt.axis('equal')
            plt.tight_layout()
            plt.show()

# Create and display the calculator
calculator = PerovskiteCalculator()
display(calculator.grid)

# Run the calculation once to show initial results
calculator.calculate(None)


GridspecLayout(children=(HTML(value='<h1 style="text-align: center;">Perovskite Solution Calculator</h1>', lay…